In [ ]:
import onnx
import onnxruntime
import  matplotlib.pyplot as plt
import cv2
import numpy as np

In [ ]:
try:
    onnx.checker.check_model('C:/Users/stan/Desktop/yolov8/runs/detect/train19/weights/best.onnx')
except onnx.checker.ValidationError as e:
    print('invalid')
else:
    print('valid')

In [ ]:
onnx_file_name='C:/Users/stan/Desktop/yolov8/runs/detect/train19/weights/best.onnx'
ort_session=onnxruntime.InferenceSession(onnx_file_name,providers=['CUDAExecutionProvider'])


In [ ]:
for input in ort_session.get_inputs():
    print("input name: ", input.name)
    print("input shape: ", input.shape)
    print("input type: ", input.type)

for output in ort_session.get_outputs():
    print("output name: ", output.name)
    print("output shape: ", output.shape)
    print("output type: ", output.type)

In [ ]:
def prepare_input(bgr_image, width, height):
    image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (width, height)).astype(np.float32)
    image = image / 255.0
    image = np.transpose(image, (2, 0, 1))
    input_tensor = np.expand_dims(image, axis=0)
    return input_tensor

In [ ]:
image = cv2.imread("yolo_dataset/images/val/val1001.jpg")
image_height, image_width, _ = image.shape
input_tensor = prepare_input(image, 640,640)
print("input_tensor shape: ", input_tensor.shape)

In [ ]:
outputs = ort_session.run(None, {ort_session.get_inputs()[0].name: input_tensor})

In [ ]:
output0=np.squeeze(outputs[0]).transpose()

In [ ]:
output0.shape

In [ ]:
def nms(pred, conf_thres, iou_thres): 
    conf = pred[..., 4] > conf_thres
    box = pred[conf == True] 
    cls_conf = box[..., 5:]
    cls = []
    for i in range(len(cls_conf)):
        cls.append(int(np.argmax(cls_conf[i])))
    total_cls = list(set(cls))
    # print(total_cls)  
    output_box = []  
    for i in range(len(total_cls)):
        clss = total_cls[i] 
        cls_box = []
        for j in range(len(cls)):
            if cls[j] == clss:
                box[j][5] = clss
                cls_box.append(box[j][:6])
        cls_box = np.array(cls_box)
        box_conf = cls_box[..., 4]  
        box_conf_sort = np.argsort(box_conf) 
        max_conf_box = cls_box[box_conf_sort[len(box_conf) - 1]]
        output_box.append(max_conf_box) 
        cls_box = np.delete(cls_box, box_conf_sort[len(box_conf) - 1], 0) 
        while len(cls_box) > 0:
            max_conf_box = output_box[len(output_box) - 1]  
            del_index = []
            for j in range(len(cls_box)):
                current_box = cls_box[j]  
                interArea = getInter(max_conf_box, current_box)  
                iou = getIou(max_conf_box, current_box, interArea)  
                if iou > iou_thres:
                    del_index.append(j)  
            cls_box = np.delete(cls_box, del_index, 0)  
            if len(cls_box) > 0:
                box_conf = cls_box[..., 4]
                box_conf_sort = np.argsort(box_conf) 
                max_conf_box = cls_box[box_conf_sort[len(box_conf) - 1]]
                output_box.append(max_conf_box)
                cls_box = np.delete(cls_box, box_conf_sort[len(box_conf) - 1], 0)
    return output_box

In [ ]:
def getIou(box1, box2, inter_area):
    box1_area = box1[2] * box1[3]
    box2_area = box2[2] * box2[3]
    union = box1_area + box2_area - inter_area
    iou = inter_area / union
    return iou

In [ ]:
def getInter(box1, box2):
    box1_x1, box1_y1, box1_x2, box1_y2 = box1[0] - box1[2] / 2, box1[1] - box1[3] / 2, \
                                         box1[0] + box1[2] / 2, box1[1] + box1[3] / 2
    box2_x1, box2_y1, box2_x2, box2_y2 = box2[0] - box2[2] / 2, box2[1] - box1[3] / 2, \
                                         box2[0] + box2[2] / 2, box2[1] + box2[3] / 2
    if box1_x1 > box2_x2 or box1_x2 < box2_x1:
        return 0
    if box1_y1 > box2_y2 or box1_y2 < box2_y1:
        return 0
    x_list = [box1_x1, box1_x2, box2_x1, box2_x2]
    x_list = np.sort(x_list)
    x_inter = x_list[2] - x_list[1]
    y_list = [box1_y1, box1_y2, box2_y1, box2_y2]
    y_list = np.sort(y_list)
    y_inter = y_list[2] - y_list[1]
    inter = x_inter * y_inter
    return inter

In [ ]:
def draw(img, xscale, yscale, pred):
    img_ = img.copy()
    if len(pred):
        for detect in pred:
            detect = [int((detect[0] - detect[2] / 2) * xscale), int((detect[1] - detect[3] / 2) * yscale),
                      int((detect[0]+detect[2] / 2) * xscale), int((detect[1]+detect[3] / 2) * yscale)]
            img_ = cv2.rectangle(img, (detect[0], detect[1]), (detect[2], detect[3]), (0, 255, 0), 1)
    return img_

In [ ]:
pred_class = output0[..., 4:]
print(pred_class[0])
pred_conf = np.max(pred_class, axis=-1)
pred = np.insert(output0, 4, pred_conf, axis=-1)
print(pred[0])
result = nms(pred, 0.5, 0.5)
ret_img = draw(image, image.shape[1]/640, image.shape[0]/640, result)
ret_img = ret_img[:, :, ::-1]
plt.imshow(ret_img)
plt.show()